This notebook is a Google Colab adaptation of Cysmith's "neural-style-tf" repository on Github (https://github.com/cysmith/neural-style-tf). 

The model implements Neural Style Transfer in Tensorflow to create artistic images and videos. 

The purpose of this notebook is to leverage Google's free GPU runtimes and Colab collaboration tools. It was created on July 22nd, 2018 by Jimmy Lin

#Basic Codes that need to run every runtime

连接Colab和Google Drive，使之能互相访问

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#执行上面的代码过程中，会出来两次认证，每次都是点击链接后允许，然后把代码复制回Colab运行窗口中的文本框中回车。

In [0]:
!mkdir -p /content/drive
!google-drive-ocamlfuse drive #其实这就是在虚拟机里创建了一个网络硬盘并映射到Drive这个目录
import os
os.chdir("drive") # 指定drive为当前的工作文件夹

指定程序运行主目录

In [0]:
main_dir='/content/drive/neural-style-tf'

# Basic Codes that only need to run once
(store all docs in google drive permanently)

获取Neural-Style-tf的全部源代码和文件夹结构，保存到你的Google Drive上，这样即使Colab清空虚拟机后，下次你打开本脚本也不用重新下载了

In [0]:
!git clone https://github.com/cysmith/neural-style-tf

用我修改过的neural_style.py 替换原始版本，主要是Frame文件格式的修改以适应超过1000帧的视频的转换

In [0]:
%cd $main_dir
%mv neural_style.py neural_style_original.py
!wget https://github.com/ljm2020/Neural-Style-TF-Application/blob/master/neural_style.py

下载VGG-19训练好的模型文件

In [0]:
%cd $main_dir
!wget http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat

# Image Transfer Codes

检查文件夹结构，如有错误，重新执行上面的代码

In [0]:
!ls $main_dir

In [0]:
!pip install colorama

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

选择要转换的文件

In [0]:
#Content
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

%cd $main_dir/image_input/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Content Image ({Fore.RED}One Image Only!!!{Fore.BLUE}):{Style.RESET_ALL}")
uploaded = files.upload()

Content_Img_list=[]
i=0
for fn in uploaded.keys():
  print('User uploaded content image(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Content_Img_list.append(fn)
  i+=1
  
if len(Content_Img_list)==0:
  print(f"{Style.BRIGHT}{Fore.RED}ERROR!!! No image file uploaded! please re-run this cell!!{Style.RESET_ALL}")
elif len(Content_Img_list)==1:
  Content_Img=Content_Img_list[0]
  img=mpimg.imread(Content_Img)
  imgplot = plt.imshow(img)
  plt.title(Content_Img)
  plt.grid(False)
  plt.axis('off')
  plt.show()
  Content_Img='"'+Content_Img+'"'
else:
  print(f"{Style.BRIGHT}{Fore.RED}ERROR!!! only one image should be uploaded! all uploaded file deleted!!! please re-run this cell!!{Style.RESET_ALL}")
  i=0
  while i<len(Content_Img_list):
    filename_list=os.path.splitext(Content_Img_list[i])
    fn_del='"'+filename_list[0]
    ext_del=filename_list[1]+'"'
    %rm $fn_del$ext_del
    i+=1
  


选择风格文件

In [0]:
#Style Image Upload

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

Style_Img = []
Style_Img_Path = []

%cd $main_dir/styles/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Style Image(s): (One or More Images):{Style.RESET_ALL}")
uploaded = files.upload()
i=0

for fn in uploaded.keys():
  print('User uploaded style image(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Style_Img.append(fn)
  Style_Img_Path.append(os.path.abspath(fn))
  i+=1
  
j=0
for j in range(i):
  img=mpimg.imread(Style_Img[j])
  imgplot = plt.imshow(img)
  plt.title(Style_Img[j])
  plt.grid(False)
  plt.axis('off')
  plt.show()
  Style_Img[j]='"'+Style_Img[j]+'"'
  j+=1

检查风格文件和待转换文件名，方便下面调用

In [0]:
print(Content_Img)
print(Style_Img)

执行单风格文件转换

In [0]:
%cd $main_dir

!python neural_style.py --content_img {Content_Img} \
                       --style_imgs {Style_Img[0]} \
                       --max_size 300 \
                       --max_iterations 300 \
                       --device /gpu:0 \
                       --verbose;
                       #--original_colors \

img=mpimg.imread(main_dir+'/image_output/result/result.png')
imgplot = plt.imshow(img)
plt.title('Converted Output')
plt.axis('off')
plt.grid(False)
plt.show() 

或 执行多风格混合转化

In [0]:
%cd $main_dir

!python neural_style.py --content_img {Content_Img} \
                       --style_imgs {Style_Img[0]} {Style_Img[1]} {Style_Img[2]} \
                       --style_imgs_weights 1 1 1 \
                       --max_size 666 \
                       --max_iterations 300 \
                       --device /gpu:0 \
                       --verbose;
                       #--original_colors \

img=mpimg.imread(main_dir+'/image_output/result/result.png')
imgplot = plt.imshow(img)
plt.title('Converted Output')
plt.grid(False)
plt.axis('off')
plt.show() 

或 执行带Mask的风格转换（Mask图片中白色的区域会有风格变化，黑色区域没有变化）

单风格或多风格都适用


In [0]:
#Style Masks Upload

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

%cd $main_dir/image_input/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Style Mask Image(s): (One or More Images):{Style.RESET_ALL}")
uploaded = files.upload()
i=0
Style_Mask=['','','','','','']
for fn in uploaded.keys():
  print('User uploaded content image(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Style_Mask[i]=fn
  i+=1
j=0
for j in range(i):
  img=mpimg.imread(Style_Mask[j])
  imgplot = plt.imshow(img)
  plt.title(Style_Mask[j])
  plt.grid(False)
  plt.show()
  Style_Mask[j]='"'+Style_Mask[j]+'"'
  j+=1

检查风格文件和Mask文件名，方便下面一一对应

In [0]:
print(Content_Img)
print(Style_Mask)
print(Style_Img)

In [0]:
%cd $main_dir
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#要注意风格图片和Mask的对应关系，因为上传后List里面的顺序完全是系统自动按照字母数字来排序的，
#可能不是一一对应，需要自行判断！

!python neural_style.py --content_img {Content_Img} \
                       --style_imgs {Style_Img[0]} {Style_Img[1]} \ #风格文件顺序
                       --style_imgs_weights 1 1 \ #风格文件权重分配
                       --style_mask \
                       --style_mask_imgs {Style_Mask[1]} {Style_Mask[0]} \ #Mask文件顺序，确保一一对应
                       --max_size 1000 \
                       --max_iterations 500 \
                       --device /gpu:0 \
                       --verbose;
                       #--original_colors \ #是否只转化风格而保留原图颜色

img=mpimg.imread(main_dir+'/image_output/result/result.png')
imgplot = plt.imshow(img)
plt.title('Converted Output')
plt.grid(False)
plt.axis('off')
plt.show() 

最后：下载转换完成的图像文件

In [0]:
from google.colab import files
files.download('/content/neural-style-tf/image_output/result/result.png')

# Video Transfer Codes

安装ffmpeg视频图片转换包（每天运行一次，因为Colab虚拟机一段时间后会给你清空内容，而这个ffmpeg只能装到系统里，所以会根着一起被清空）

In [0]:
!apt-get install ffmpeg

定义文件查找函数，后面用得着

In [0]:
import os

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

选择要转换的源视频

In [0]:
#Content
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

%cd $main_dir/video_input/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Content Video ({Fore.RED}One Video Only!!!{Fore.BLUE}):{Style.RESET_ALL}")
uploaded = files.upload()

Content_Vid_list=[]
i=0
for fn in uploaded.keys():
  print('User uploaded content video(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Content_Vid_list.append(os.path.abspath(fn))
  i+=1
  
if len(Content_Vid_list)==0:
  print(f"{Style.BRIGHT}{Fore.RED}ERROR!!! No video file uploaded! please re-run this cell!!{Style.RESET_ALL}")
elif len(Content_Vid_list)==1:
  Content_Vid_Path=Content_Vid_list[0]
  Content_Vid_Path='"'+Content_Vid_Path+'"'
else:
  print(f"{Style.BRIGHT}{Fore.RED}ERROR!!! only one video should be uploaded! all uploaded file deleted!!! please re-run this cell!!{Style.RESET_ALL}")
  i=0
  while i<len(Content_Vid_list):
    bn_del=!basename $Content_Vid_list[i]
    %rm $bn_del
    i+=1

选择风格图片 及/或 Mask（一个或多个）

In [0]:
#Style Image Upload

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

Style_Img = []
Style_Img_Path = []

%cd $main_dir/styles/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Style Image(s): (One or More Images):{Style.RESET_ALL}")
uploaded = files.upload()
i=0

for fn in uploaded.keys():
  print('User uploaded style image(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Style_Img.append(fn)
  Style_Img_Path.append(os.path.abspath(fn))
  i+=1
  
j=0
for j in range(i):
  img=mpimg.imread(Style_Img[j])
  imgplot = plt.imshow(img)
  plt.title(Style_Img[j])
  plt.grid(False)
  plt.axis('off')
  plt.show()
  Style_Img[j]='"'+Style_Img[j]+'"'
  j+=1

In [0]:
#Style Masks Upload

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from colorama import Fore
from colorama import Style

%cd $main_dir/image_input/

from google.colab import files
print(f"{Style.BRIGHT}{Fore.BLUE}Upload Style Mask Image(s): (One or More Images):{Style.RESET_ALL}")
uploaded = files.upload()
i=0
Style_Mask=[]
for fn in uploaded.keys():
  print('User uploaded style mask image(s) "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  Style_Mask.append(fn)
  i+=1
  
j=0
for j in range(i):
  img=mpimg.imread(Style_Mask[j])
  imgplot = plt.imshow(img)
  plt.title(Style_Mask[j])
  plt.grid(False)
  plt.show()
  Style_Mask[j]='"'+Style_Mask[j]+'"'
  j+=1

In [0]:
#print(Content_Vid_Path)
print(Style_Mask)
print(Style_Img)

视频转换代码变量定义及赋值

In [0]:
%cd $main_dir
content_video=Content_Vid_Path

content_dir_list=!dirname $content_video
content_dir=content_dir_list[0]

content_filename_ext=!basename $content_video
content_filename_list=os.path.splitext(content_filename_ext[0])
content_filename=content_filename_list[0]
extension=content_filename_list[1]

style_image=Style_Img_Path[0]
style_dir_list=!dirname $style_image
style_dir=style_dir_list[0]

style_filename_list=!basename $style_image
style_filename=style_filename_list[0]

print(content_dir, content_filename)
print(style_dir, style_filename)

!mkdir -p /content/video_input

temp_dir="/content/video_input/"+content_filename
!mkdir -p $temp_dir
!ls $temp_dir


将视频导出成帧图片存放于临时目录，并定义输出尺寸

In [0]:
original_size=!ffprobe -v error -of flat=s=_ -select_streams v:0 -show_entries stream=width,height $content_video
original_width=original_size[0][original_size[0].find('=')+1:]
original_height=original_size[1][original_size[1].find('=')+1:]
print(original_width,original_height)

max_size=500
if original_width >= original_height:
  out_width=int(max_size)
  out_height=int(int(out_width)*int(original_height)/int(original_width))
else:
  out_height=int(max_size)
  out_width=int(int(out_height)*int(original_width)/int(original_height))
print(out_width,out_height)

size=str(int(out_width))+"x"+str(int(out_height))
# Save frames of the video as individual image files
#%cd $main_dir
!ffmpeg -v quiet -i $content_video  -s $size $temp_dir/frame_%08d.ppm 

In [0]:
!ls $temp_dir/

检查导出的图像帧数

In [0]:
num_frames_list=!find $temp_dir -iname "*.ppm" | wc -l
num_frames=num_frames_list[0]
print(num_frames)

开始对每帧图像进行风格化转换并保存至视频输出目录，之后将风格化后的帧图片合成视频文件

In [0]:
!echo "Computing optical flow [CPU]. This will take a while..."
%cd $main_dir/video_input

filePattern=temp_dir+"/frame_"
folderName=temp_dir
startFrame=1
stepSize=1

i=startFrame
j=startFrame+stepSize


def ppm_rename(index):
  filename=str(index).zfill(8)+".ppm"
  return filename

#while j <=int(num_frames):
while j <=100:
  file1=filePattern+ppm_rename(i)
  file2=filePattern+ppm_rename(j)

  print(file1, file2)

  forward_name=folderName+"/forward_"+str(i)+"_"+str(j)+".flo"
  #print(forward_name)

  if(find("forward_"+str(i)+"_"+str(j)+".flo",temp_dir)==None):
          !$main_dir/video_input/deepmatching-static $file1 $file2 -nt 0 | $main_dir/video_input/deepflow2-static $file1 $file2 $forward_name -match
  
  backward_name=folderName+"/backward_"+str(j)+"_"+str(i)+".flo"
  #print(backward_name)
  if(find("backward_"+str(j)+"_"+str(i)+".flo",temp_dir)==None):
    !$main_dir/video_input/deepmatching-static $file2 $file1 -nt 0 | $main_dir/video_input/deepflow2-static $file2 $file1 $backward_name -match
 
  forward_reliable_txt=folderName+"/reliable_"+str(i)+"_"+str(j)+".txt"
  backward_reliable_txt=folderName+"/reliable_"+str(j)+"_"+str(i)+".txt"
  if(find("reliable_"+str(j)+"_"+str(i)+".txt",temp_dir)==None):
     !$main_dir/video_input/consistencyChecker/consistencyChecker $backward_name $forward_name $backward_reliable_txt
  if(find("reliable_"+str(i)+"_"+str(j)+".txt",temp_dir)==None):
     !$main_dir/video_input/consistencyChecker/consistencyChecker $forward_name $backward_name $forward_reliable_txt

  i+=1
  j+=1

j=j-1
print (j)

%cd $main_dir

!echo "Rendering stylized video frames [CPU & GPU]. This will take a while..."
!python neural_style.py --video \
                       --video_input_dir $temp_dir \
                       --video_output_dir $main_dir/video_output/$content_filename \
                       --style_imgs_dir $style_dir \
                       --style_imgs $style_filename \
                       --start_frame $startFrame \
                       --end_frame $j \
                       --max_size $max_size \
                       --first_frame_iterations 300\
                       --frame_iterations 200;
                       #--verbose;

# Create video from output images.
!echo "Converting image sequence to video.  This should be quick..."
out_put_name=main_dir+"/video_output/"+content_filename+"/"+content_filename+"-stylized.mp4"
!ffmpeg -i $main_dir/video_output/$content_filename/frame_%08d.ppm \
                       -i $main_dir/video_input/$content_filename$extension \
                       -map 0:v -map 1:a -c:a copy -shortest \
                       -pix_fmt yuv420p \
                       -vf pad="width=ceil(iw/2)*2:height=ceil(ih/2)*2" \
                       $out_put_name;

In [0]:
!ffmpeg -i $main_dir/video_output/$content_filename/frame_%08d.ppm \
                       -i $main_dir/video_input/$content_filename$extension \
                       -map 0:v -map 1:a -c:a copy -shortest \
                       -pix_fmt yuv420p \
                       -vf pad="width=ceil(iw/2)*2:height=ceil(ih/2)*2" \
                       $out_put_name;

将转换完成的视频下载到本地

In [0]:
from google.colab import files
files.download(out_put_name)

清理临时文件夹（否则下次转换其它文件会出错！）

In [0]:
# Clean up garbage
%rm -rf $temp_dir